In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
%matplotlib widget
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.pyplot as plt
import chemiscope
from widget_code_input import WidgetCodeInput
from ipywidgets import Textarea
from iam_utils import *
import ase
from ase.io import read, write

In [ ]:
#### AVOID folding of output cell 

In [ ]:
%%html

<style>
.output_wrapper, .output {
    height:auto !important;
    max-height:4000px;  /* your desired max-height here */
}
.output_scroll {
    box-shadow:none !important;
    webkit-box-shadow:none !important;
}
</style>

In [ ]:
data_dump = WidgetDataDumper(prefix="ex_04")
display(data_dump)

_Reference textbook / figure credits: Allen, Tildesley, Computer simulations of liquids, (2017), Chapter 1_

# Interatomic potentials

Interatomic potentials describe the energy (~stability) of a set of atoms - characterized by their chemical nature $a_i$ and Cartesian coordinates $\mathbf{r}_i$ - in terms of a model that describes their interactions. This potential $V(\{\mathbf{r}_i\})$ can be seen as an approximation of the quantum mechanical energy of the electrons in a material or molecule for a given position of the nuclei (the so-called Born-Oppenheimer approximation).

Many empirical forms have been proposed to model the interatomic potential. A typical potential might look something like

$$
V(\{\mathbf{r}_i\}) = \sum_{ij} \frac{Z_i Z_j}{|\mathbf{r}_i - \mathbf{r}_j|} - \sum_{ij} \frac{A}{|\mathbf{r}_i - \mathbf{r}_j|^6} + k \sum_{i,j \in \mathrm{bonds}} k (|\mathbf{r}_i - \mathbf{r}_j| - r_0)^2
$$

where you may recognize an electrostatic term, a dispersion interaction, and harmonic bonds that are usually chosen as a simple model of covalent bonds. 

Coulomb and dispersion forces are usually referred to as _non-bonded_ terms, in that they act between all pairs of atoms of a given kind. Harmonic springs terms (or angles, or dihedrals) are _bonded_ terms, that only act between selected groups of atoms that are chosen based on a predetermined topology of the covalent bonds.

All of the terms above are _pair potentials_ i.e. functions of just the distance between pairs of atoms. More complicated functional forms exist, as we shall see later.

<span style="color:blue">**01** How does the energy of a non-bonded and a bonded term in the potential change as the separation between two atoms tends to infinity? Can a harmonic bond ever truly dissociate? </span>

In [ ]:
ex01_txt = Textarea("enter your answer", layout=Layout(width="100%"))
data_dump.register_field("ex01-answer", ex01_txt, "value")
display(ex01_txt)

An archetypal example of a non-bonded potential is the Lennard-Jones potential (if you are curious, you can read [the paper in which the general functional form was proposed](https://doi.org/10.1098/rspa.1924.0081)).
The LJ potential is a non-bonded pair potential $V(r)$ in which the attractive and repulsive parts are both algebraic functions of the interatomic separation, $A/r^m-B/r^n$. Usually $1/r^6$ is used for the attractive part (that physically corresponds to dispersion/van der Waals forces), and $1/r^{12}$ for the repulsive parts (which is chosen just to have a steep repulsive wall, and because back in the old days you could compute this just by squaring $1/r^6$, which was cheaper than recomputing another power. 

You can experiment below with the more general form of the potential,
$$
V(r) = \frac{A}{r^m} - \frac{B}{r^n}
$$
See how exponents and prefactors change the shape of the curve.

In [ ]:
def plot_LJ(ax, A, B, m, n, x_max = 3, y_min_relative = -1.5, y_max_relative = 2, n_points = 200):
    
    if (m != n):
        # min_pos and min_energy are max pos and max energy when n > m
        min_pos = np.exp((np.log(A) - np.log(B) + np.log(m) - np.log(n))/(m - n))
        min_energy = A / (min_pos ** m) - B / (min_pos ** n)
        min_energy = np.abs(min_energy)

        y_min = min_energy * y_min_relative
        y_max = min_energy * y_max_relative
    else:
        y_min, y_max = y_min_relative, y_max_relative
        
    grid = np.linspace(0, x_max, 200)[1:] # excluding 0
    curve = A / (grid ** m) - B / (grid ** n)
    
    ax.plot(grid, curve, color = 'red', linewidth = 2)
    ax.set_title(r"$V(r) = \frac{A}{r^m} - \frac{B}{r^n}$", fontsize = 15)
    ax.set_xlim([0, x_max])
    ax.set_ylim([y_min, y_max])
    ax.set_xlabel("r", fontsize = 15)
    ax.set_ylabel("V(r)", fontsize = 15)
    
    ax.tick_params(axis='both', which='major', labelsize=12)
    ax.tick_params(axis='both', which='minor', labelsize=12)
A = WidgetPlot(plot_LJ, WidgetParbox(A = (1.0, 0.0, 10, 0.1, r'A'),
                                       B = (1.0, 0.0, 10, 0.1, r'B'),
                                       m = (12, 1, 20, 1, r'm'),
                                       n = (6, 1, 20, 1, r'n'),
                                       ))
display(A)

The more common form to express the LJ potential is

$$
V(r) = 4\epsilon \left((\frac{\sigma}{r})^12 - (\frac{\sigma}{r})^6\right).
$$

<span style="color:blue">**02** Compute analytically the equilibrium separation $r_0$ between two atoms (i.e. the position of the minimum in the $V(r)$ curve. What is the corresponding energy? </span>

In [ ]:
ex02_txt = Textarea("enter your answer", layout=Layout(width="100%"))
data_dump.register_field("ex02-answer", ex02_txt, "value")
display(ex02_txt)

<span style="color:blue">**03** Now consider a set of four atoms arranged as a square with side $a$. Write a function that computes the total LJ potential for this structure, as a function of $a$. Inspect the curve as a function of $a$, using the sliders to select an appropriate range.</span>

_Take for simplicity $\epsilon=1$ and $\sigma=1$ (which is equivalent to writing the problem in natural units. You can write the summation as a sum over the pair distances, without writing explicitly the position of the particles._

In [ ]:
# set upt the code widget window
ex03_wci = WidgetCodeInput(
        function_name="total_LJ_square", 
        function_parameters="a",
        docstring="""
Computes the total LJ potential for the structure of four atoms arranged as a square with side a. 

:param a: side of the square
        
:return: the value of the total energy
""",
        function_body="""
# Write your solution. You can use np.sqrt(2) to get the value of sqrt(2)
# Note you can define a function inside a function body - use this to also write
# a function that computes the LJ potential at a given distance
import numpy as np

def compute_LJ(r):
    # computes the value of LJ potential depending on the distance r
    # use epsilon=sigma=1
    return ...
    
total_energy = 0.0  # write here a sum over the various interactions

return total_energy
"""
        )

data_dump.register_field("ex03-function", ex03_wci, "function_body")

def plot_total_energy(ax, x_min, x_max, y_min, y_max, n_points = 200):
    grid = np.linspace(x_min, x_max, n_points)[1:]
    func = ex03_wci.get_function_object()    
    values = [func(x) for x in grid]
    ax.plot(grid, values, color = 'red', linewidth = 2)
    ax.set_xlim(x_min, x_max)
    ax.set_ylim(y_min, y_max)
    ax.set_xlabel("a", fontsize = 15)
    ax.set_ylabel("total energy", fontsize = 15)
    
    ax.tick_params(axis='both', which='major', labelsize=12)
    ax.tick_params(axis='both', which='minor', labelsize=12)
    
ex03_plot = WidgetPlot(plot_total_energy, WidgetParbox(x_min = (0.1, 0, 1.0, 0.1, r'$x_{min}$'),
                                                        x_max = (4.0, 1.0, 10, 0.1, r'$x_{max}$'),
                                       y_min = (-1.0, -10, 0, 0.1, r'$y_{min}$'),
                                       y_max = (2.0, 0, 10, 0.1, r'$y_{max}$'),
                                       ));
ref_val = np.linspace(0.2, 5, 10)
ref_nrg = [3936501953.1249976,
 557.321470897532,
 -3.1707345581849036,
 -0.4858813626814373,
 -0.1047196369691888,
 -0.030580375933941077,
 -0.010997872358105667,
 -0.004589582164219913,
 -0.002140404149784412,
 -0.0010879339519999998]
ex03_ref_values = {(val,) : nrg for val, nrg in zip(ref_val, ref_nrg) }
ex03_wcc = WidgetCodeCheck(ex03_wci, ref_values = ex03_ref_values, demo=ex03_plot)    
display(ex03_wcc)

<span style="color:blue">**04** Is the equilibrium separation between the particles the same as that which minimizes the energy of a dimer? Write both the analytical expression, finding the minimum of the total energy that contains all interactions as a function of the square side $a$. Compare it with the plot above.</span>

In [ ]:
ex04_txt = Textarea("enter your answer", layout=Layout(width="100%"))
data_dump.register_field("ex04-answer", ex04_txt, "value")
display(ex04_txt)

# Locality, cutoffs and minimum-image convention

One typical problem one encounters is that non-bonded potentials must (in principles) be evaluated among _all pairs of atoms_. This means that the computational effort grows as $N_{\text{atoms}}^2$ for a finite structure. But what about a _periodic_ structure? Then one would need to sum over multiple cells, making the effort essentially infinite!

In practice this is a real issue only for electrostatic interactions (for which [solutions](https://en.wikipedia.org/wiki/Ewald_summation) exist, but are much too complicated for this introductory course). For other long-range terms one usually artificially makes the interaction zero beyond a selected _cutoff_ distance $r_\text{cut}$. This is an approximation, but hardly the worst one we are making - the functional form of the potential is an approximation anyway. 

This below is a cluster of rare-gas atoms (which are well modeled by a LJ potential). If you click on an atom, it will highlight the atoms within the selected cutoff distance. Experiment with it to get a feel of the range of the interactions and how many atoms are actually included when you select different values for the cutoff. 

In [ ]:
lj55 = read('data/lj-structures.xyz',":1")

properties = {}
cs = chemiscope.show(lj55, mode="structure",                      
                     environments=chemiscope.all_atomic_environments(lj55),
                     settings={"structure":[{"bonds":True, "unitCell":False,
                                            "environments": {"cutoff": 3}}]}                    
                    )

def update_co(change):
    cs.settings={"structure": [{"environments": {"cutoff": pb.value['co']}}]}
pb = WidgetParbox(onchange=update_co, co=(3.,1,5,0.25, r"environment cutoff / Å"))
display(VBox([pb,cs]))

<span style="color:blue">**05** Write a function that loops over all pairs of atoms in this icosahedral cluster, and computes a LJ potential (with unit $\epsilon$ and $\sigma$). Only compute the potential for $r_{ij}<r_\mathrm{cut}$. Observe the plot demonstrating the convergence of the total energy. </span>

In [ ]:
# TODO SP: Make a code widget where they have to compute this with a double loop, 
# and that shows as a demo a curve of the total energy as a function of the cutoff

Now let's do the same for a bulk sample. Even if the cutoff is finite, one may have to sum over multiple copies of the supercell to account for all the interactions that contribute to the energy of the periodic solid. If however the supercell is large enough to contain a sphere of size $r_\text{cut}$ (informally, if it is at least $2r_\text{cut}$ in every direction) one can utilize a more efficient scheme, the _minimal image convention_.

Essentially, one would only let the interaction loop run over the atoms in the supercell, and for each pair consider the periodic replicas with the smallest possible separation. 

# TESTS & CRAP

In [ ]:
frame = ase.Atoms("He2")

In [ ]:
frame.positions[:,0] = [-1,1]

In [ ]:
from ase.calculators import lj, eam

In [ ]:
ljcalc = lj.LennardJones(sigma=1.0, epsilon=1.0)

In [ ]:
frame.calc = ljcalc
frame.get_potential_energy()
frame.get_forces()

In [ ]:
frame.info

In [ ]:
from ase.build import bulk
frame = bulk('Al', 'fcc', a=4.05, cubic=True)

In [ ]:
eamcalc = eam.EAM(potential='data/Al99.eam.alloy')

In [ ]:
frame.calc = eamcalc
frame.get_potential_energy()

# x